In [1]:
import random
import numpy as np
from collections import deque
import sys
import os
import multiprocessing
import random
from collections import Counter

In [2]:
from deap import base
from deap import creator
from deap import tools
from scoop import futures

In [3]:
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [4]:
IND_ROWS = 8
IND_COLS = 14
IND_SIZE = IND_ROWS * IND_COLS
INT_MIN, INT_MAX = 0, 9
toolbox = base.Toolbox()

In [5]:
def count_occurrences(grid, n):
    S = str(n)
    target_len = len(S)
    digits = [int(d) for d in S]
    count = 0
    deltas = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

    def is_valid(r, c):
        return 0 <= r < IND_ROWS and 0 <= c < IND_COLS

    def dfs(r, c, idx):
        nonlocal count
        if idx == target_len - 1:
            count += 1
            return

        if count > 500: return

        next_digit = digits[idx + 1]
        for dr, dc in deltas:
            nr, nc = r + dr, c + dc
            if is_valid(nr, nc) and grid[nr][nc] == next_digit:
                dfs(nr, nc, idx + 1)

    starts = [(r, c) for r in range(IND_ROWS) for c in range(IND_COLS) if grid[r][c] == digits[0]]
    if not starts: return 0
    if target_len == 1: return 1

    for r, c in starts:
        dfs(r, c, 0)
        if count > 500: return count
    return count

In [6]:
def eval_814_heuristic(individual):
    grid = np.array(individual).reshape(IND_ROWS, IND_COLS)
    total_heuristic = 0.0
    n = 1
    MAX_N = 50000

    while n < MAX_N:
        M = count_occurrences(grid, n)
        if M == 0:
            X = n - 1
            break

        if n < 10:
            total_heuristic += 1.0
        else:
            total_heuristic += 1.0 / M

        n += 1
    else:
        X = MAX_N - 1

    return X, total_heuristic

In [7]:
def custom_mutate(individual, indpb):
    digit_counts = Counter(individual)

    indpb_uniform = indpb
    indpb_swp = 0.1
    
    max_digit = max(digit_counts, key=digit_counts.get)
    total_len = len(individual)
    avg_count = total_len / 10.0
    under_avg_digits = [d for d in range(10) if digit_counts[d] < avg_count]
   
    if len(under_avg_digits) >= 4:
        locations = [i for i, d in enumerate(individual) if d == max_digit]
        for pos in locations:
            if random.random() < indpb_uniform:
                individual[pos] = under_avg_digits[random.randrange(len(under_avg_digits))]
    else:
        for _ in range(total_len):
            if random.random() < indpb_swp:
                pos1 = random.randint(0, total_len - 2)
                pos2 = random.randint(0, total_len - 2)
                if pos1 == pos2:
                    pos2 += 1
                individual[pos1], individual[pos2] = individual[pos2], individual[pos1]
                
    return individual,

In [8]:
toolbox.register("evaluate", eval_814_heuristic)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", custom_mutate, indpb=0.7)
toolbox.register("select", tools.selTournament, tournsize=3)

toolbox.register("attr_int", random.randint, INT_MIN, INT_MAX)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_int, IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [9]:
filename = 'best_output_double.txt'
def main():
    loaded = []
    protected = None
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            valid_lines = []
            for line in f:
                stripped = line.strip()
                if len(stripped) == 14 and stripped.isdigit():
                    valid_lines.append(stripped)
    
        for block_start in range(0, len(valid_lines), 8):
            if block_start + 7 >= len(valid_lines):
                break
            block = valid_lines[block_start:block_start + 8]
        
        ind_list = []
        for row in block:
            ind_list.extend(int(d) for d in row)
        
        if len(ind_list) == IND_SIZE:
            ind = creator.Individual(ind_list)
            if block_start == 0:
                protected = ind
            else:
                loaded.append(ind)
                
    pop = []
    if protected is not None:
        pop.append(protected)
    pop.extend(loaded)
    target_pop_size = 300
    remaining = target_pop_size - len(pop)
    if remaining > 0:
        pop.extend(toolbox.population(n=remaining))
    if len(pop) > target_pop_size:
        pop = pop[:target_pop_size]
        
    with multiprocessing.Pool() as pool:
        toolbox.register("map", pool.map)
        fitnesses = list(toolbox.map(toolbox.evaluate, pop))
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
        CXPB, MUTPB = 0.3, 0.5
        g = 0
        while g < 100:
            g += 1
            offspring = toolbox.select(pop, len(pop))
            offspring = list(map(toolbox.clone, offspring))
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < CXPB:
                    toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values
            for mutant in offspring:
                if random.random() < MUTPB:
                    toolbox.mutate(mutant)
                    del mutant.fitness.values
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = list(toolbox.map(toolbox.evaluate, invalid_ind))
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
            pop[:] = offspring
            fits = [ind.fitness.values[0] for ind in pop]

            if g % 100 == 0:
                length = len(pop)
                mean = sum(fits) / length
                sys.stdout.write(f"-- Generation {g} --\n")
                sys.stdout.write(f" Max Score: {max(fits):.4f}\n")
                sys.stdout.write(f" Avg Score: {mean:.4f}\n")
        best_ind = tools.selBest(pop, 1)[0]
        sys.stdout.write(f"\nBest individual Score: {best_ind.fitness.values[0]:.4f}\n")
        grid = np.array(best_ind).reshape(IND_ROWS, IND_COLS)
        
        for row in grid:
            sys.stdout.write(''.join(map(str, row)) + '\n')
        flat = list(best_ind)
        sys.stdout.write("\nDigit Counts:\n")
        for i in range(10):
            sys.stdout.write(f"{i}: {flat.count(i)} ")
        sys.stdout.write("\n")
        with open(filename, 'a') as f:
            grid = np.array(best_ind).reshape(IND_ROWS, IND_COLS)
            for row in grid:
                f.write(''.join(map(str, row)) + '\n')
            f.write('\n')

In [10]:
if __name__ == "__main__":
    main()


-- Generation 100 --
 Max Score: 767.0000
 Avg Score: 313.4500

Best individual Score: 767.0000
73464346900965
05883677627117
01983921556589
69192535605360
98028630429162
41190290281544
58306481247337
74857340782495

Digit Counts:
0: 12 1: 10 2: 10 3: 11 4: 12 5: 12 6: 12 7: 10 8: 11 9: 12 
